## Trying to go over medium code and fix it

In [1]:
import asyncio # to run async methods and tool calls
import os
import urllib.parse
from dotenv import load_dotenv
from datetime import datetime

from google.adk.artifacts import InMemoryArtifactService # short term storage for files/artifacts created during runs
from google.adk.runners import Runner # engine that feeds messages through your agent graph and yields events
from google.adk.sessions import InMemorySessionService # tracks conversation state and returns a session object (async)
from google.genai import types as genai_types # message structures
from google.adk.agents import LlmAgent # adk wrapper around llm and tools

from google.adk.auth import AuthConfig, AuthCredential, AuthCredentialTypes, OAuth2Auth
# AuthCredential / OAuth2Auth: carry client id/secret and dynamic fields
from google.adk.tools.openapi_tool.openapi_spec_parser.openapi_toolset import OpenAPIToolset
# parses your OpenAPI specs, returns ADK tool objects
from fastapi.openapi.models import OAuth2, OAuthFlowAuthorizationCode, OAuthFlows

c:\Users\User\anaconda3\envs\PT\Lib\site-packages\pydantic\_internal\_fields.py:198: UserWarning: Field name "config_type" in "SequentialAgent" shadows an attribute in parent "BaseAgent"
  warnings.warn(


In [ ]:
client_id = "609528807091-1jslq73gobk3pii89u8v3km7ife7epli.apps.googleusercontent.com"
client_secret = "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"

In [ ]:
MODEL_NAME = Config.MODEL_NAME
REDIRECT_URI = Config.REDIRECT_URI

In [ ]:
required_scopes = {
    # "https://www.googleapis.com/auth/gmail.readonly": "gmail read scope",
    # "https://www.googleapis.com/auth/gmail.send": "gmail send scope",
    # "https://www.googleapis.com/auth/userinfo.profile": "user profile scope",
    "https://www.googleapis.com/auth/calendar.events": "calendar read/write events scope",
    "https://www.googleapis.com/auth/calendar.calendarlist": "calendar read scope"
}
auth_scheme = OAuth2(
   flows=OAuthFlows(
      authorizationCode=OAuthFlowAuthorizationCode(
            authorizationUrl="https://accounts.google.com/o/oauth2/auth",
            tokenUrl="https://oauth2.googleapis.com/token",
            scopes=required_scopes,
      )
   )
)
# auth_scheme describes the OAuth2 flow to the toolset (how to authorize & get tokens).
auth_credential = AuthCredential(
   auth_type=AuthCredentialTypes.OAUTH2,
   oauth2=OAuth2Auth(
      client_id=client_id,
      client_secret=client_secret,
      redirect_uri=REDIRECT_URI,
   ),
)

In [2]:
# from google.adk.tools.google_api_tool import CalendarToolset
# cal = CalendarToolset(
#     client_id="609528807091-1jslq73gobk3pii89u8v3km7ife7epli.apps.googleusercontent.com",
#     client_secret="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",
# )


Step 1 :Run Agent & Detect Auth Request

Initiate the agent interaction using runner.run_async.
Iterate through the yielded events.
Look for a specific function call event whose function call has a special name: adk_request_credential. This event signals that user interaction is needed. You can use helper functions to identify this event and extract necessary information. (For the second case, the logic is similar. You deserialize the event from the http response).

In [ ]:
from google.adk.events import Event
from google.adk.auth import AuthConfig # Import necessary type
from google.genai import types

def get_auth_request_function_call(event: Event) -> types.FunctionCall:
    # Get the special auth request function call from the event
    if not event.content or event.content.parts:
        return
    for part in event.content.parts:
        if (
            part 
            and part.function_call 
            and part.function_call.name == 'adk_request_credential'
            and event.long_running_tool_ids 
            and part.function_call.id in event.long_running_tool_ids
        ):

            return part.function_call

def get_auth_config(auth_request_function_call: types.FunctionCall) -> AuthConfig:
    # Extracts the AuthConfig object from the arguments of the auth request function call
    if not auth_request_function_call.args or not (auth_config := auth_request_function_call.args.get('auth_config')):
        raise ValueError(f'Cannot get auth config from function call: {auth_request_function_call}')
    if not isinstance(auth_config, AuthConfig):
        raise ValueError(f'Cannot get auth config {auth_config} is not an instance of AuthConfig.')
    return auth_config

In [ ]:
print("\nRunning agent...")
events_async = runner.run_async(
    session_id=session.id, user_id='user', new_message=content
)

auth_request_function_call_id, auth_config = None, None

async for event in events_async:
    # Use helper to check for the specific auth request event
    if (auth_request_function_call := get_auth_request_function_call(event)):
        print("--> Authentication required by agent.")
        # Store the ID needed to respond later
        if not (auth_request_function_call_id := auth_request_function_call.id):
            raise ValueError(f'Cannot get function call id from function call: {auth_request_function_call}')
        # Get the AuthConfig containing the auth_uri etc.
        auth_config = get_auth_config(auth_request_function_call)
        break # Stop processing events for now, need user interaction

if not auth_request_function_call_id:
    print("\nAuth not required or agent finished.")
    # return # Or handle final response if received

Step 2: Redirect User for Authorization

Get the authorization URL (auth_uri) from the auth_config extracted in the previous step.
Crucially, append your application's redirect_uri as a query parameter to this auth_uri. This redirect_uri must be pre-registered with your OAuth provider (e.g., Google Cloud Console, Okta admin panel).
Direct the user to this complete URL (e.g., open it in their browser).

In [ ]:
# (Continuing after detecting auth needed)

if auth_request_function_call_id and auth_config:
    # Get the base authorization URL from the AuthConfig
    base_auth_uri = auth_config.exchanged_auth_credential.oauth2.auth_uri

    if base_auth_uri:
        redirect_uri = 'http://localhost:8000/callback' # MUST match your OAuth client app config
        # Append redirect_uri (use urlencode in production)
        auth_request_uri = base_auth_uri + f'&redirect_uri={redirect_uri}'
        # Now you need to redirect your end user to this auth_request_uri or ask them to open this auth_request_uri in their browser
        # This auth_request_uri should be served by the corresponding auth provider and the end user should login and authorize your applicaiton to access their data
        # And then the auth provider will redirect the end user to the redirect_uri you provided
        # Next step: Get this callback URL from the user (or your web server handler)
    else:
         print("ERROR: Auth URI not found in auth_config.")
         # Handle error

Step 3. Handle the Redirect Callback (Client):

Your application must have a mechanism (e.g., a web server route at the redirect_uri) to receive the user after they authorize the application with the provider.
The provider redirects the user to your redirect_uri and appends an authorization_code (and potentially state, scope) as query parameters to the URL.
Capture the full callback URL from this incoming request.
(This step happens outside the main agent execution loop, in your web server or equivalent callback handler.)



Step 4. Send Authentication Result Back to ADK (Client):

Once you have the full callback URL (containing the authorization code), retrieve the auth_request_function_call_id and the auth_config object saved in Client Step 1.
Set the captured callback URL into the exchanged_auth_credential.oauth2.auth_response_uri field. Also ensure exchanged_auth_credential.oauth2.redirect_uri contains the redirect URI you used.
Create a types.Content object containing a types.Part with a types.FunctionResponse.
Set name to "adk_request_credential". (Note: This is a special name for ADK to proceed with authentication. Do not use other names.)
Set id to the auth_request_function_call_id you saved.
Set response to the serialized (e.g., .model_dump()) updated AuthConfig object.
Call runner.run_async again for the same session, passing this FunctionResponse content as the new_message.


In [ ]:
# (Continuing after user interaction)

    # Simulate getting the callback URL (e.g., from user paste or web handler)
    auth_response_uri = await get_user_input(
        f'Paste the full callback URL here:\n> '
    )
    auth_response_uri = auth_response_uri.strip() # Clean input

    if not auth_response_uri:
        print("Callback URL not provided. Aborting.")
        return

    # Update the received AuthConfig with the callback details
    auth_config.exchanged_auth_credential.oauth2.auth_response_uri = auth_response_uri
    # Also include the redirect_uri used, as the token exchange might need it
    auth_config.exchanged_auth_credential.oauth2.redirect_uri = redirect_uri

    # Construct the FunctionResponse Content object
    auth_content = types.Content(
        role='user', # Role can be 'user' when sending a FunctionResponse
        parts=[
            types.Part(
                function_response=types.FunctionResponse(
                    id=auth_request_function_call_id,       # Link to the original request
                    name='adk_request_credential', # Special framework function name
                    response=auth_config.model_dump() # Send back the *updated* AuthConfig
                )
            )
        ],
    )

    # --- Resume Execution ---
    print("\nSubmitting authentication details back to the agent...")
    events_async_after_auth = runner.run_async(
        session_id=session.id,
        user_id='user',
        new_message=auth_content, # Send the FunctionResponse back
    )

    # --- Process Final Agent Output ---
    print("\n--- Agent Response after Authentication ---")
    async for event in events_async_after_auth:
        # Process events normally, expecting the tool call to succeed now
        print(event) # Print the full event for inspection

Step 5: ADK Handles Token Exchange & Tool Retry and gets Tool result

ADK receives the FunctionResponse for adk_request_credential.
It uses the information in the updated AuthConfig (including the callback URL containing the code) to perform the OAuth token exchange with the provider's token endpoint, obtaining the access token (and possibly refresh token).
ADK internally makes these tokens available by setting them in the session state).
ADK automatically retries the original tool call (the one that initially failed due to missing auth).
This time, the tool finds the valid tokens (via tool_context.get_auth_response()) and successfully executes the authenticated API call.
The agent receives the actual result from the tool and generates its final response to the user.

### Note

Tried desperately, did not get results as expected. trying a very different new way where i do my own tools from scratch, this way i can benefit from better deployment functionalities

ref: https://google.github.io/adk-docs/tools/authentication/#2-handling-the-interactive-oauthoidc-flow-client-side

ref: https://medium.com/google-cloud/building-a-multi-agent-application-to-interact-with-google-products-b7ff7eb2f17d

## Trying to work around the authentication trouble

In [32]:
import datetime
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/calendar.readonly"]

# event checker
def scrape_calendar():
  """Shows basic usage of the Google Calendar API.
  Prints the start and name of the next 10 events on the user's calendar.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
      creds = flow.run_local_server(host="localhost", port=50759)  # -> http://localhost:50759/
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

  try:
    service = build("calendar", "v3", credentials=creds)

    # Call the Calendar API
    now = datetime.datetime.now(tz=datetime.timezone.utc).isoformat()
    print("Getting the upcoming 10 events")
    events_result = (
        service.events()
        .list(
            calendarId="primary",
            timeMin=now,
            maxResults=10,
            singleEvents=True,
            orderBy="startTime",
        )
        .execute()
    )
    events = events_result.get("items", [])

    if not events:
      print("No upcoming events found.")
      return

    # Prints the start and name of the next 10 events
    for event in events:
      print(event)
      start = event["start"].get("dateTime", event["start"].get("date"))
      print(start, event["summary"])
      print('eventId:',event["id"])

  except HttpError as error:
    print(f"An error occurred: {error}")


In [53]:
scrape_calendar()

Getting the upcoming 10 events
{'kind': 'calendar#event', 'etag': '"3511118841344702"', 'id': '3i9f0qtgu2u86v3f2fv5b9de0g', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=M2k5ZjBxdGd1MnU4NnYzZjJmdjViOWRlMGcgZXk4MTUxODYwQG0', 'created': '2025-08-18T23:23:40.000Z', 'updated': '2025-08-18T23:23:40.672Z', 'summary': 'Dinner with friends', 'creator': {'email': 'ey8151860@gmail.com', 'self': True}, 'organizer': {'email': 'ey8151860@gmail.com', 'self': True}, 'start': {'dateTime': '2025-08-23T05:00:00+03:00', 'timeZone': 'Asia/Beirut'}, 'end': {'dateTime': '2025-08-23T08:00:00+03:00', 'timeZone': 'Asia/Beirut'}, 'iCalUID': '3i9f0qtgu2u86v3f2fv5b9de0g@google.com', 'sequence': 0, 'reminders': {'useDefault': True}, 'eventType': 'default'}
2025-08-23T05:00:00+03:00 Dinner with friends
eventId: 3i9f0qtgu2u86v3f2fv5b9de0g
{'kind': 'calendar#event', 'etag': '"3504849786880574"', 'id': '6ti3gob564sjab9n6lh3eb9k74r6cb9o70oj6b9o68q32oppc8pmao9p6s', 'status': 'confirmed', 

## Agent auth function

this will be added in langgraph before calling the adk main agent

In [ ]:
from pathlib import Path

SCOPES = ["https://www.googleapis.com/auth/gmail.compose"]
HERE = Path(__file__).resolve().parent
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
print('path: ',HERE)
if os.path.exists(Path(f"HERE/tokern.json")):
    creds = Credentials.from_authorized_user_file(f"{HERE}/token.json", SCOPES)
    # If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(f"{HERE}/credentials.json", SCOPES)
        creds = flow.run_local_server(host="localhost", port=50759)  # http://localhost:50759/

with open(f"{HERE}/token.json", "w", encoding="utf-8") as f:
    f.write(creds.to_json())


## Get Events

In [ ]:
def scrape_calendar():
  """Shows basic usage of the Google Calendar API.
    Prints the start and name of the next 10 events on the user's calendar.
  """
#   creds = None
#   # The file token.json stores the user's access and refresh tokens, and is
#   # created automatically when the authorization flow completes for the first
#   # time.
#   if os.path.exists("token.json"):
#     creds = Credentials.from_authorized_user_file("token.json", SCOPES)
#   # If there are no (valid) credentials available, let the user log in.
#   if not creds or not creds.valid:
#     if creds and creds.expired and creds.refresh_token:
#       creds.refresh(Request())
#     else:
#       flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
#       creds = flow.run_local_server(host="localhost", port=50759)  # -> http://localhost:50759/
#     # Save the credentials for the next run
#     with open("token.json", "w") as token:
#       token.write(creds.to_json())

  try:
    service = build("calendar", "v3", credentials=creds)

    # Call the Calendar API
    now = datetime.datetime.now(tz=datetime.timezone.utc).isoformat()
    print("Getting the upcoming 10 events")
    events_result = (
        service.events()
        .list(
            calendarId="primary",
            timeMin=now,
            maxResults=10,
            singleEvents=True,
            orderBy="startTime",
        )
        .execute()
    )
    events = events_result.get("items", [])

    if not events:
      print("No upcoming events found.")
      return

    # Prints the start and name of the next 10 events
    for event in events:
      print(event)
      start = event["start"].get("dateTime", event["start"].get("date"))
      print(start, event["summary"])
      print('eventId:',event["id"])

  except HttpError as error:
    print(f"An error occurred: {error}")


## Insert event

In [ ]:
import datetime
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/calendar"]

# event checker
def insert_to_calendar():
  """Shows basic usage of the Google Calendar API.
  Prints the start and name of the next 10 events on the user's calendar.
  """
  # creds = None
  # # The file token.json stores the user's access and refresh tokens, and is
  # # created automatically when the authorization flow completes for the first
  # # time.
  # if os.path.exists("token.json"):
  #   creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # # If there are no (valid) credentials available, let the user log in.
  # if not creds or not creds.valid:
  #   if creds and creds.expired and creds.refresh_token:
  #     creds.refresh(Request())
  #   else:
  #     flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
  #     creds = flow.run_local_server(host="localhost", port=50759)  # -> http://localhost:50759/
  #   # Save the credentials for the next run
  #   with open("token.json", "w") as token:
  #     token.write(creds.to_json())
  try:
    service = build("calendar", "v3", credentials=creds)


    GMT_OFF = '-07:00'  # PDT/MST/GMT-7
    EVENT = {
        'summary': 'Dinner with friends',
        'start': {'dateTime': '2025-08-22T19:00:00%s' % GMT_OFF},
        'end':   {'dateTime': '2025-08-22T22:00:00%s' % GMT_OFF},
        # 'attendees': [
        #     # {'email': 'friend@example.com'},
        # ],
    }

    event = service.events().insert(calendarId='primary', body=EVENT).execute()

    print('''*** %r event added:
    Start: %s
    End:   %s''' % (event['summary'].encode('utf-8'),
                    event['start']['dateTime'], event['end']['dateTime']))

  except HttpError as error:
        print(f"An error occurred: {error}")


In [52]:
insert_to_calendar()

*** b'Dinner with friends' event added:
    Start: 2025-08-23T05:00:00+03:00
    End:   2025-08-23T08:00:00+03:00


## Update Calendar

In [ ]:
import datetime
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/calendar"]

# event checker
def update_calendar():
  """Shows basic usage of the Google Calendar API.
  Prints the start and name of the next 10 events on the user's calendar.
  """
  # creds = None
  # # The file token.json stores the user's access and refresh tokens, and is
  # # created automatically when the authorization flow completes for the first
  # # time.
  # if os.path.exists("token.json"):
  #   creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # # If there are no (valid) credentials available, let the user log in.
  # if not creds or not creds.valid:
  #   if creds and creds.expired and creds.refresh_token:
  #     creds.refresh(Request())
  #   else:
  #     flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
  #     creds = flow.run_local_server(host="localhost", port=50759)  # -> http://localhost:50759/
  #   # Save the credentials for the next run
  #   with open("token.json", "w") as token:
  #     token.write(creds.to_json())
  try:
    service = build("calendar", "v3", credentials=creds)


    # GMT_OFF = '-07:00'  # PDT/MST/GMT-7
    # EVENT = {
    #     'summary': 'Dinner with friends',
    #     'start': {'dateTime': '2025-08-22T19:00:00%s' % GMT_OFF},
    #     'end':   {'dateTime': '2025-08-22T22:00:00%s' % GMT_OFF},
    #     # 'attendees': [
    #     #     # {'email': 'friend@example.com'},
    #     # ],
    # }
    GMT_OFF = '-07:00'  # PDT/MST/GMT-7

    updated_event_body = {
    'summary': 'Updated Meeting Title',
    'location': 'New Meeting Room',
    'description': 'This is an updated description for the meeting.',
    'start': {'dateTime': '2025-08-23T19:00:00%s' % GMT_OFF},
    'end':   {'dateTime': '2025-08-23T22:00:00%s' % GMT_OFF},
    }

    eventId= "3i9f0qtgu2u86v3f2fv5b9de0g"
    updated_event= service.events().update(calendarId='primary',
                                          eventId=eventId,
                                          body=updated_event_body,
                                          sendUpdates='all' # 'all', 'externalOnly', or 'none'
                                          ).execute()
    print(f"Event updated: {updated_event['htmlLink']}")

  except HttpError as error:
        print(f"An error occurred: {error}")



In [61]:
update_calendar()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=609528807091-1jslq73gobk3pii89u8v3km7ife7epli.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A50759%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=OzzgGHvtNAzvVY0kGLxjmWUMHnfu6X&access_type=offline
Event updated: https://www.google.com/calendar/event?eid=M2k5ZjBxdGd1MnU4NnYzZjJmdjViOWRlMGcgZXk4MTUxODYwQG0


## Delete From Calendar

In [ ]:
import datetime
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/calendar"]

# event checker
def delete_from_calendar():
  """Shows basic usage of the Google Calendar API.
  Prints the start and name of the next 10 events on the user's calendar.
  """
  # creds = None
  # # The file token.json stores the user's access and refresh tokens, and is
  # # created automatically when the authorization flow completes for the first
  # # time.
  # if os.path.exists("token.json"):
  #   creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # # If there are no (valid) credentials available, let the user log in.
  # if not creds or not creds.valid:
  #   if creds and creds.expired and creds.refresh_token:
  #     creds.refresh(Request())
  #   else:
  #     flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
  #     creds = flow.run_local_server(host="localhost", port=50759)  # -> http://localhost:50759/
  #   # Save the credentials for the next run
  #   with open("token.json", "w") as token:
  #     token.write(creds.to_json())
  try:
    service = build("calendar", "v3", credentials=creds)


    # GMT_OFF = '-07:00'  # PDT/MST/GMT-7
    # EVENT = {
    #     'summary': 'Dinner with friends',
    #     'start': {'dateTime': '2025-08-22T19:00:00%s' % GMT_OFF},
    #     'end':   {'dateTime': '2025-08-22T22:00:00%s' % GMT_OFF},
    #     # 'attendees': [
    #     #     # {'email': 'friend@example.com'},
    #     # ],
    # }
    eventId= "c5tv80indvprfq2v09r0v28dhs"
    event= service.events().delete(calendarId='primary',eventId=eventId ).execute()

  except HttpError as error:
        print(f"An error occurred: {error}")



In [49]:
delete_from_calendar()

## Draft Email TOOL

In [ ]:
# from __future__ import annotations
# import base64
# import mimetypes
# import os
# from email.message import EmailMessage
# from typing import Any, Dict, List, Optional
# from mcp.server.fastmcp import FastMCP
# from googleapiclient.discovery import build
# from google.oauth2.credentials import Credentials
# from google_auth_oauthlib.flow import InstalledAppFlow
# from google.auth.transport.requests import Request
# from googleapiclient.errors import HttpError

# SCOPES = ["https://www.googleapis.com/auth/gmail.compose"]

# creds = None
#   # The file token.json stores the user's access and refresh tokens, and is
#   # created automatically when the authorization flow completes for the first
#   # time.
# if os.path.exists("token.json"):
#     creds = Credentials.from_authorized_user_file("token.json", SCOPES)
#   # If there are no (valid) credentials available, let the user log in.
# if not creds or not creds.valid:
#     if creds and creds.expired and creds.refresh_token:
#       creds.refresh(Request())
#     else:
#       flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
#       creds = flow.run_local_server(host="localhost", port=50759)  # http://localhost:50759/
#     # Save the credentials for the next run
#     with open("token.json", "w") as token:
#       token.write(creds.to_json())

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=609528807091-1jslq73gobk3pii89u8v3km7ife7epli.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A50759%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.compose&state=fuc6FHpX8H0MFoJUSZCd8xozSf3SLk&access_type=offline


In [ ]:
# with open("token.json", "w", encoding="utf-8") as f:
#     f.write(creds.to_json())

In [ ]:
# service =build("gmail", "v1", credentials=creds)

In [ ]:
# service

In [78]:
import base64
from email.message import EmailMessage

import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


def gmail_create_draft(to: str, body: str, meeting_date: datetime = datetime.datetime.now() , subject: str ="Tutoring Session"):
  """Create and insert a draft email.
   Print the returned draft's message and id.
   Returns: Draft object, including draft id and message meta data.

  Load pre-authorized user credentials from the environment.
  TODO(developer) - See https://developers.google.com/identity
  for guides on implementing OAuth2 for the application.
  
Args:
    to (str): Recipient email address (RFC 5322 format).
    body (str): Plain-text email content (e.g., the planned session outline).
    meeting_date (datetime.datetime): The meeting’s date/time. If timezone-aware,
        it will be formatted accordingly; if naive, treat as UTC unless your
        implementation defines otherwise.

Returns:
    dict: Gmail `Draft` resource as returned by the API, e.g.
        `{"id": "<draft_id>", "message": {...}}`.

Raises:
    ValueError: If inputs are missing or invalid (e.g., empty `to` or `body`).
    googleapiclient.errors.HttpError: If the Gmail API request fails.

Notes:
    - This function only creates a draft; sending the email is a separate action.
    - Subject line and body formatting are implementation-defined.
  """
  try:
    # create gmail api client
    service = build("gmail", "v1", credentials=creds)

    message = EmailMessage()
    
    time_str = meeting_date.strftime("%H:%M")

    content="""
    Greetings,
    
    I hope this email finds you well,
    
    Note that we will have our meeting on """+time_str+""" , to access the meeting session, please use the following https://www.youtube.com/ ,
    
    For our session, we will cover the following topics:
      
    """+ body+"""
    
    Best regards,
    Ali
    """
    
    message.set_content(content)

    message["To"] = to
    message["From"] = "ey8151860@gmail.com"
    message["Subject"] = subject

    # encoded message
    encoded_message = base64.urlsafe_b64encode(message.as_bytes()).decode()

    create_message = {"message": {"raw": encoded_message}}
    # pylint: disable=E1101
    draft = (
        service.users()
        .drafts()
        .create(userId="me", body=create_message)
        .execute()
    )

    print(f'Draft id: {draft["id"]}\nDraft message: {draft["message"]}')

  except HttpError as error:
    print(f"An error occurred: {error}")
    draft = None

  return draft


In [79]:
res = gmail_create_draft(
    to="ay8151863@gmail.com",           # or "a@x.com, b@y.com"
    subject="Test draft from API",
    body="This is a test draft created via Gmail API.",
    # sender="me"                           # or a specific alias you own
)
print(res)


Draft id: r-4389168225113594754
Draft message: {'id': '198c13cce15fe10e', 'threadId': '198c13cce15fe10e', 'labelIds': ['DRAFT']}
{'id': 'r-4389168225113594754', 'message': {'id': '198c13cce15fe10e', 'threadId': '198c13cce15fe10e', 'labelIds': ['DRAFT']}}


#### Loading the api

In [71]:
from dotenv import load_dotenv
import os

load_dotenv(".env", override=True)
os.environ["GOOGLE_CSE_ID"] = os.getenv("GOOGLE_CSE_ID")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

#### Trying with the agent

In [73]:
gmail_agent = LlmAgent(
    model='gemini-2.0-flash',
    name="google_gmail_agent",
    description="Handles Gmail tasks like drafting emails.",
    instruction=f"You handle queries related to Gmail. Do not ask any followup questions related to user ids, gmail ids etc. You don't need to know the actual email address or user ID if you're making requests on behalf of the logged-in user. Use the available tools to fulfill the user's request. If you encounter an error, provide the *exact* error message so the user can debug.",
    tools=[gmail_create_draft]
) if gmail_create_draft else None

In [ ]:
session_service_stateful = InMemorySessionService()
